In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from collections import defaultdict
from datetime import datetime 
from tqdm import tqdm_notebook
from collections import Counter
from bs4 import BeautifulSoup as bs
from tqdm import tqdm_notebook
from selenium.common.exceptions import NoSuchElementException

import urllib.request
import urllib.parse
import lxml.html
import requests
import pandas as pd
import time
import math
import re
import random

# word2vec
from gensim.models.word2vec import Word2Vec
import gensim 
import gensim.models as g

In [185]:
# driver path
path = 'C:/Users/Administrator/chromedriver'
driver = webdriver.Chrome(path)

In [2]:
# url 데이터프레임 생성
urlDf = pd.DataFrame(data = {'유튜버': ['디렉터파이','디렉터파이','디렉터파이','디렉터파이','디렉터파이','이사배','디렉터파이','회사원a','회사원a','회사원a'],
                              '제목': ['(*Eng) 내 피부가 이랬어?! 민감, 지성, 건성 에센스 BEST 궁합 찾는 법 by 디렉터파이',
                                    '[디파GO]민감 건성, 보습과 탄력? 화장품 바르는 4가지 방법(디렉터파이 피부 측정 결과?)',
                                     '(**Eng.)여드름, 좁쌀, 각질 부자 모여! 피부타입별 성분 궁합& 착한 크림들끼리 대격돌, 과연 승자는? by 디렉터파이',
                                     '(*Eng)피부타입별 토너 사용법 따로 있다? 효과 검증 화장솜 궁합까지. by 디렉터파이',
                                     '속건조 해결?! 지성, 수부지 민감한 피부 위한 기초 스킨케어 by. 디렉터파이',
                                     '[Eng] 민감피부🤦🏻‍♀ 나의 하루종일 피부진정방법🍀 Calming Skincare Routine l 이사배(RISABAE Makeup)',
                                     '(*Eng) 국내 더마 코스메틱 Top5 성분 효과 검증 by 디렉터파이',
                                     '민감피부 되돌리기 3일 프로젝트',
                                     '살아남아! 이 각박한 민감피부 권하는 세상에서!! feat.더마토리',
                                     '기초 바르는 방법만 바꿔서 피부 탄력 살리기!! feat.밀도케어'
                                    ],
                              'url': ['https://www.youtube.com/watch?v=YkX9SA166RM&t=51s',
                                     'https://www.youtube.com/watch?v=6zti2UXhQ3A',
                                     'https://www.youtube.com/watch?v=q12lCuRff34',
                                      'https://www.youtube.com/watch?v=C8wXn0Xdp1c',
                                      'https://www.youtube.com/watch?v=N1M2ocbcHb0',
                                      'https://www.youtube.com/watch?v=xsBKNABqi5E',
                                      'https://www.youtube.com/watch?v=_sejSV5yXro',
                                      'https://www.youtube.com/watch?v=Wf90OkNhAGY&t=3s',
                                      'https://www.youtube.com/watch?v=K0Q6-r1aHRE',
                                      'https://www.youtube.com/watch?v=8aBYjw2C8us'
                                     ],
                              '조회수': [317550,
                                      183588,
                                      1432136,
                                      961828,
                                      943392,
                                      456728,
                                      184022,
                                      1416944,
                                      167386,
                                      674600
                                      
                                     ],
                              '날짜':['2018-06-08',
                                   '2017-09-01',
                                   '2017-10-20',
                                    '2018-05-04',
                                   '2017-08-11',
                                   '2018-06-09',
                                   '2018-10-29',
                                   '2016-12-28',
                                   '2019-03-17',
                                   '2019-01-22'],
                              '댓글수': [302,
                                     177,
                                     1080,
                                      849,
                                     786,
                                     426,
                                     282,
                                      1383,
                                     297, 
                                      976]})

In [3]:
urlDf

,유튜버,제목,url,조회수,날짜,댓글수
0,디렉터파이,"(*Eng) 내 피부가 이랬어?! 민감, 지성, 건성 에센스 BEST 궁합 찾는 법...",https://www.youtube.com/watch?v=YkX9SA166RM&t=51s,317550,2018-06-08,302
1,디렉터파이,"[디파GO]민감 건성, 보습과 탄력? 화장품 바르는 4가지 방법(디렉터파이 피부 측...",https://www.youtube.com/watch?v=6zti2UXhQ3A,183588,2017-09-01,177
2,디렉터파이,"(**Eng.)여드름, 좁쌀, 각질 부자 모여! 피부타입별 성분 궁합& 착한 크림들...",https://www.youtube.com/watch?v=q12lCuRff34,1432136,2017-10-20,1080
3,디렉터파이,(*Eng)피부타입별 토너 사용법 따로 있다? 효과 검증 화장솜 궁합까지. by 디...,https://www.youtube.com/watch?v=C8wXn0Xdp1c,961828,2018-05-04,849
4,디렉터파이,"속건조 해결?! 지성, 수부지 민감한 피부 위한 기초 스킨케어 by. 디렉터파이",https://www.youtube.com/watch?v=N1M2ocbcHb0,943392,2017-08-11,786
5,이사배,[Eng] 민감피부🤦🏻‍♀ 나의 하루종일 피부진정방법🍀 Calming Skincar...,https://www.youtube.com/watch?v=xsBKNABqi5E,456728,2018-06-09,426
6,디렉터파이,(*Eng) 국내 더마 코스메틱 Top5 성분 효과 검증 by 디렉터파이,https://www.youtube.com/watch?v=_sejSV5yXro,184022,2018-10-29,282
7,회사원a,민감피부 되돌리기 3일 프로젝트,https://www.youtube.com/watch?v=Wf90OkNhAGY&t=3s,1416944,2016-12-28,1383
8,회사원a,살아남아! 이 각박한 민감피부 권하는 세상에서!! feat.더마토리,https://www.youtube.com/watch?v=K0Q6-r1aHRE,167386,2019-03-17,297
9,회사원a,기초 바르는 방법만 바꿔서 피부 탄력 살리기!! feat.밀도케어,https://www.youtube.com/watch?v=8aBYjw2C8us,674600,2019-01-22,976


In [4]:
urlDf.to_excel('피부진정,민감피부 컨텐츠.xlsx')

## 잠재고객 url 리스트 만들기

In [183]:
len(urlDf.iloc[7:,])

3

In [188]:
#reply_cnt = int(driver.find_element_by_css_selector('#count > yt-formatted-string').text[3:-1]) #댓글수
user_list = []
reply_list = []

for i in range(7,10):
    driver.get(urlDf.url[i])
    reply_cnt = urlDf.댓글수[i]

    start = datetime.now()
    batang = driver.find_element_by_css_selector("body") 
    while True : 
        now = datetime.now() 

        batang.send_keys(Keys.PAGE_DOWN)  
        time.sleep(1)

        # 더보기 클릭하는 반복문
        try:
            more_btns = driver.find_elements_by_xpath('//*[@id="more-replies"]')
            for more_btn in more_btns:
                try:
                    more_btn.click()
                    time.sleep(1)
                except:
                    pass            
        except:
            pass

        replys = driver.find_elements_by_css_selector('yt-formatted-string#content-text.style-scope.ytd-comment-renderer')
        check_cnt = len(replys) 
        if check_cnt == reply_cnt:
            break
        elif (now-start).seconds > 300:
            break
    
    time.sleep(2)

    # 댓글 단 사람들의 url
    user_urls = driver.find_elements_by_css_selector('#author-text')  
    time.sleep(2)
    for user_url in user_urls:
        user_list.append([user_url.text, user_url.get_attribute('href')])

    # 댓글 단 사람들의 댓글
    replys = driver.find_elements_by_css_selector('yt-formatted-string#content-text.style-scope.ytd-comment-renderer')
    time.sleep(2)
    for reply in replys:
        reply_list.append(reply.text)
        
    continue

In [189]:
replyDf = pd.DataFrame(reply_list, columns = ['댓글내용'])
replyDf.head()

,댓글내용
0,"민감할땐 팩하면 안 돼요..제일 심할땐 병원가서 한 3일은 약 먹고 바르시구요, 모..."
1,0:40썬글라스쫌 오래쓰셨나보네...
2,전 일어나면 얼굴이 무거워질 정도로 기름이 올라와서...
3,대신 답변 죄송하지만 저도 그랬는데 그럴때마다 아침 세안을 열심히 해주는 것이 중요...
4,@몽럠ᄒ 네 그래서 항상 세안 열심히 하고 있어요 ㅎㅎ


In [190]:
len(replyDf)

2019

In [192]:
replyDf.to_excel('유튜브댓글(회사원추가).xlsx')

In [193]:
replyDf = pd.read_excel('유튜브댓글(회사원추가).xlsx', index_col=0)
replyDf.head()

,댓글내용
0,"민감할땐 팩하면 안 돼요..제일 심할땐 병원가서 한 3일은 약 먹고 바르시구요, 모..."
1,0:40썬글라스쫌 오래쓰셨나보네...
2,전 일어나면 얼굴이 무거워질 정도로 기름이 올라와서...
3,대신 답변 죄송하지만 저도 그랬는데 그럴때마다 아침 세안을 열심히 해주는 것이 중요...
4,@몽럠ᄒ 네 그래서 항상 세안 열심히 하고 있어요 ㅎㅎ


## 댓글 분석

In [213]:
replyDf1 = pd.read_excel('유튜브댓글(회사원추가).xlsx', index_col=0)
replyDf1.head()

,댓글내용
0,"안녕하세요, 오늘 올라갈 영상은 내 피부타입 아는 법-기초(1편 에센스), (2편 ..."
1,디렉터 파이 Do english subtitle please~🙏🏻
2,겨울버전도 해주세요ㅠㅠ
3,코랑 이마엔 기름이많고 다른곳은 건조하면 수부지인가요?
4,디렉터 파이 토너나 다른 것들은 안바르고 에센스 수분크림만 발라도될까요?? 고수님들...


In [211]:
replyDf.head()

,댓글내용
0,"민감할땐 팩하면 안 돼요..제일 심할땐 병원가서 한 3일은 약 먹고 바르시구요, 모..."
1,0:40썬글라스쫌 오래쓰셨나보네...
2,전 일어나면 얼굴이 무거워질 정도로 기름이 올라와서...
3,대신 답변 죄송하지만 저도 그랬는데 그럴때마다 아침 세안을 열심히 해주는 것이 중요...
4,@몽럠ᄒ 네 그래서 항상 세안 열심히 하고 있어요 ㅎㅎ


In [220]:
replyTot = pd.concat([replyDf1,replyDf], axis=0)
replyTot.head()

,댓글내용
0,"안녕하세요, 오늘 올라갈 영상은 내 피부타입 아는 법-기초(1편 에센스), (2편 ..."
1,디렉터 파이 Do english subtitle please~🙏🏻
2,겨울버전도 해주세요ㅠㅠ
3,코랑 이마엔 기름이많고 다른곳은 건조하면 수부지인가요?
4,디렉터 파이 토너나 다른 것들은 안바르고 에센스 수분크림만 발라도될까요?? 고수님들...


In [58]:
from worin.tag import Worin

In [221]:
tagger= Worin()
def get_nouns(text):
    try: # 에러가 발생하지 않을 경우
        nouns = tagger.nouns(text)
        nouns = [word for word in nouns if len(word) > 1]
        return nouns
    except: # 에러 발생
        return ["error"]

In [222]:
## CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=1000, tokenizer=get_nouns)

In [223]:
## TDM
tdm = cv.fit_transform(replyTot['댓글내용'])

In [224]:
for i, n in sorted(zip(tdm[0].indices, tdm[0].data)):
    print(words[i], n)

글리세린 1
나중 2
모공 1
밀리 1
바이 1
성인 1
세범 1
순정 1
업로드 1
여드름성 1
여자 1
옛날 1
저기 1
젤크림 1
평소 1
피부관리 2
피부톤 1
피분데 2
필요 1
홍조 1
환경 1


In [225]:
## TF-IDF로 변환
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer()
tdm2 = tfidf.fit_transform(tdm)
words = cv.get_feature_names()

In [226]:
for i, n in sorted(zip(tdm2[0].indices, tdm2[0].data)):
    print(words[i], n)

궁합 0.2509862118654019
기초 0.32922908410022905
메이크업 0.19192166867018726
미백 0.23610801067113246
민감 0.13583698207152792
선크림 0.18883015682120122
성분 0.12742872899747465
수분크림 0.17234519287722158
에센스 0.18440757055380347
여름 0.1882445102256253
영상 0.10846189613380687
오늘 0.16376199447629808
장벽 0.2549524178397855
제대 0.24743831972340172
편도 0.2509862118654019
피부 0.20077526362042628
피부장벽 0.2115854308713144
피부타 0.3591776340112068
필요 0.1778391335658284
홍조 0.18654631021037205
회복 0.24129885452974917


In [227]:
from sklearn.externals import joblib

In [228]:
## 저장
from sklearn.externals import joblib
with open('dermatory_reply_tdm2_galarygirl.pkl', 'wb') as f:
    joblib.dump({'vectorizer': cv, 'tdm': tdm, 'tfidf': tdm2}, f)

In [229]:
with open('dermatory_reply_tdm2_galarygirl.pkl', 'rb') as f:
    data = joblib.load(f)
locals().update(data)

In [230]:
## 단어빈도표
wordcount = pd.DataFrame(
    {'단어': cv.get_feature_names(),
     '빈도': tdm2.sum(axis=0).flat
    }
)
wordcount.head()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\dtypes\dtypes.py:458: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if string == 'category':


,단어,빈도
0,가격,16.219409
1,가격대,2.133321
2,가능,7.006937
3,가볍,2.521762
4,가시,2.995779


In [231]:
wordcount.sort_values('빈도', ascending=False).to_excel('댓글분석(회사원추가).xlsx')

### 2. 워드임베딩

In [232]:
## Worin 형태소 분석기 사용
tagger= Worin()
def get_nouns(text):
    try: # 에러가 발생하지 않을 경우
        nouns = tagger.nouns(text)
        nouns = [word for word in nouns if len(word) > 1]
        return nouns
    except: # 에러 발생
        return ["error"]

In [233]:
documents = []
for i in range(len(replyDf.iloc[i,0])):
    documents.append(get_nouns(replyDf.iloc[i,0]))

In [234]:
documents[0]

['민감',
 '제일',
 '병원가서',
 '모델링팩',
 '마스크',
 '민감',
 '쥐약',
 '기본',
 '충실',
 '수분크림',
 '천연',
 '오일',
 '천연',
 '성분',
 '열감',
 '차갑',
 '세수',
 '마스크',
 '얼굴',
 '쥐약',
 '피부염',
 '피부',
 '혈관',
 '확장',
 '긋불',
 '고생',
 '깨달음']

In [235]:
model = Word2Vec(documents,         # 리스트 형태
                 sg=1,         # 0: CBOW, 1: Skip-gram
                 size=300,     # 벡터 크기
                 window=10,     # 고려할 앞뒤 폭(앞뒤 3단어)
                 min_count=1,  # 사용할 단어의 최소 빈도(3회 이하 단어 무시)
                 workers=4)    # 동시에 처리할 작업 수(코어 수와 비슷하게 설정)

C:\ProgramData\Anaconda3\lib\site-packages\gensim\models\base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "
W0822 17:08:54.724946   520 base_any2vec.py:1386] under 10 jobs per worker: consider setting a smaller `batch_words' for smoother alpha decay


In [236]:
len(model.wv.vocab)

317

In [237]:
model.wv.vocab

{'민감': <gensim.models.keyedvectors.Vocab at 0x18f979e8>,
 '제일': <gensim.models.keyedvectors.Vocab at 0x188bd550>,
 '병원가서': <gensim.models.keyedvectors.Vocab at 0x188bdcc0>,
 '모델링팩': <gensim.models.keyedvectors.Vocab at 0x188bdc50>,
 '마스크': <gensim.models.keyedvectors.Vocab at 0x188bddd8>,
 '쥐약': <gensim.models.keyedvectors.Vocab at 0x188bdc18>,
 '기본': <gensim.models.keyedvectors.Vocab at 0x188bd4e0>,
 '충실': <gensim.models.keyedvectors.Vocab at 0x188bdd68>,
 '수분크림': <gensim.models.keyedvectors.Vocab at 0x188bdb38>,
 '천연': <gensim.models.keyedvectors.Vocab at 0x188bdeb8>,
 '오일': <gensim.models.keyedvectors.Vocab at 0x188bd668>,
 '성분': <gensim.models.keyedvectors.Vocab at 0x188bdba8>,
 '열감': <gensim.models.keyedvectors.Vocab at 0x188bdf98>,
 '차갑': <gensim.models.keyedvectors.Vocab at 0x188bdf60>,
 '세수': <gensim.models.keyedvectors.Vocab at 0x188bdef0>,
 '얼굴': <gensim.models.keyedvectors.Vocab at 0x188bdf28>,
 '피부염': <gensim.models.keyedvectors.Vocab at 0x188bd5f8>,
 '피부': <gensim.models.k

In [ ]:
## 가장 유사한 단어
model.wv.most_similar('예민')